#Langgraph Editor Notebook

##prepare files

In [1]:
#@title clone repository
!git clone https://github.com/Erickrus/langgraph-editor

Cloning into 'langgraph-editor'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 35 (delta 14), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (35/35), 193.47 KiB | 4.84 MiB/s, done.
Resolving deltas: 100% (14/14), done.


##start service

You may need setup your ngrok account and obtain NGROK_AUTH_TOKEN and NGROK_BEARER_API_KEY

In [2]:
#@title define ngrok api
#@markdown
import json
import os
import subprocess

class Ngrok:
    # https://ngrok.com/docs/api/resources/tunnel-sessions/
    def __init__(self, authToken, bearerApiKey):
        self.authToken = authToken
        self.bearerApiKey = bearerApiKey

    def install(self):
        cmd = (
            "curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | "
            "tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && "
            'echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | '
            "tee /etc/apt/sources.list.d/ngrok.list && "
            "apt update -y && "
            "apt install -y ngrok"
        )
        try:
            subprocess.run(cmd, shell=True, check=True)
            #print("Command executed successfully.")
        except subprocess.CalledProcessError as e:
            print("Error executing command:", e)
        return self

    def authtoken(self):
        os.system("ngrok authtoken %s" % self.authToken)
        return self

    def list_session(self):
        command = [
            'curl', '-X', 'GET', '-H',
            f'Authorization: Bearer {self.bearerApiKey}',
            '-H', 'Ngrok-Version: 2',
            'https://api.ngrok.com/tunnel_sessions'
        ]
        result = subprocess.run(command, capture_output=True, text=True)
        output = json.loads(result.stdout)
        #print(output)
        session_id = ""
        try:
            session_id = output["tunnel_sessions"][0]["id"]
        except:
            pass
        return session_id

    def stop_session(self, sessionId=None):
        if type(sessionId) == type(None):
            sessionId = self.list_session()

        if sessionId == "":
            return

        command = [
            'curl', '-X', 'POST', '-H',
            f'Authorization: Bearer {self.bearerApiKey}',
            '-H', 'Content-Type: application/json',
            '-H', 'Ngrok-Version: 2',
            '-d', '{}',
            f'https://api.ngrok.com/tunnel_sessions/{sessionId}/stop'
        ]
        subprocess.run(command)

    def get_ngrok_url(self):
        try:
            cmd = "curl -s http://localhost:4040/api/tunnels 2>/dev/null"
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True, check=True)
            return json.loads(result.stdout)["tunnels"][0]["public_url"]
        except subprocess.CalledProcessError as e:
            #print("Error executing command:", e)
            return ""

from getpass import getpass

NGROK_AUTH_TOKEN = getpass(prompt="NGROK_AUTH_TOKEN:")
NGROK_BEARER_API_KEY = getpass(prompt="NGROK_BEARER_API_KEY:")
ngrok = Ngrok(NGROK_AUTH_TOKEN, NGROK_BEARER_API_KEY)
ngrok = ngrok.install().authtoken()

NGROK_AUTH_TOKEN:··········
NGROK_BEARER_API_KEY:··········


In [ ]:
#@title start tornado service
%cd /content/langgraph-editor
!nohup python3 langgraph_editor.py &
%cd /content
ngrok.stop_session()
!sleep 5
!nohup ngrok http 8082 &
!sleep 15
print(ngrok.get_ngrok_url())

##Edit langgraph

In [4]:
#@title langgraph-editor
#@markdown
from IPython.display import IFrame
import time

time.sleep(5)
iframeUrl = ngrok.get_ngrok_url() + "/static/index.html"
IFrame(src=iframeUrl, width=1080, height=600)

In [5]:
#@title show codes
#@markdown You need click menu "Compile" to get the output python code in your notebook.
#@markdown
#@markdown The output code will be stored in /content/output.py
!cat /content/output.py

from langgraph.graph import END, StateGraph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("query", query) # query
workflow.add_node("web_search", web_search) # web_search

# Build graph
workflow.add_conditional_edge(
    "query",
    decide,
    {
        "end": END, 
        "web_search": "web_search", 

    })
workflow.set_entry_point("query")

# Compile
app = workflow.compile()